In [81]:
import csv
import re
import array
import statistics
import requests
import json
import yaml
import re
import os
import networkx as nx

 Working with [Normal tissue data](http://www.proteinatlas.org/download/normal_tissue.csv.zip) from [the Human Protein Atlas version 16.1](http://www.proteinatlas.org/about)

Which is already downloaded.  
if you need to fetch you may need something like the (untried) following:   

```
    URL = 'http://www.proteinatlas.org/download/normal_tissue.csv.zip'
    rsponse = requests.get(URL)
    with open('normal_tissue.csv.zip', 'w+b') as fh
        fh.write(response.content)
 
```

or do what i did in the the shell and just issue     
```    
    wegt http://www.proteinatlas.org/download/normal_tissue.csv.zip
```

In [141]:
# what files are in this directory?
!ls -l

total 118636
-rw-r--r-- 1 tomc tomc 31109199 Jul 31 20:54 gene_tissue_curve.yaml
-rw-r--r-- 1 tomc tomc     5599 Jul 31 17:52 HGNC_symbols.ipynb
-rw-r--r-- 1 tomc tomc   983554 Jul 29 14:38 hp_atlas_coxpat.gv
-rw-r--r-- 1 tomc tomc   983555 Jul 29 14:36 hp_atlas_coxpat.gv~
-rw-r--r-- 1 tomc tomc      146 Jul 29 16:10 hp_atlas_coxpat.png
-rw-r--r-- 1 tomc tomc    73023 Aug  1 12:32 human_protien_atlas.ipynb
-rw-rw-r-- 1 tomc tomc 83451999 Jan 30  2017 normal_tissue.csv
-rw-r--r-- 1 tomc tomc  4519036 Jan 30  2017 normal_tissue.csv.zip
-rw-r--r-- 1 tomc tomc   238136 Jul 31 16:14 previous_symbol_hgnc.yaml
-rw-r--r-- 1 tomc tomc      409 Jul 26 17:44 README.human_protien_atlas


Must have also unzipped it. 

In [44]:
# need to get HGNC symbols (and NCBIGene identifiers) for these
# f(x) adapted from cq1.5
# TODO maybe check if multiple symbols are returned??? 
#      haven't seen any but I think it could happen
HGNC = 'http://rest.genenames.org/search/'
def symbol_check(string):
    response = requests.get(HGNC + string, headers={'Accept': 'application/json'})
    if response.status_code == requests.codes.ok:
        hgnc = json.loads(response.text)
        if hgnc['response']['numFound'] > 0:
            symbol = hgnc['response']['docs'][0]['symbol']
        else:
            symbol = "OBSOLETE_" + string # none found    
    else: # Make it ugly so someone will notice
        print('ERROR ' + response.url + ' returned '+ str(response.status_code))
        symbol = "BROKEN_" + string  # question broken
    return symbol 

In [46]:
s = symbol_check('BRCA1')
if s is None:
    print('None')
if s == '':
    print('Empty')   
print(s)

BRCA1


The symbol check function above works okay.  
But waiting on a API call for each gene is just wrong  
when you know you are going to ask for every one in the genome.

While waiting for those 13k http transactions to finish   
I made another notebook 'HGNC_symbols' that makes a   
local dictionary from previous symbols to current symbols  
which is loadable as a yaml file. 
process is:  
 -   check if a correction is in the dictionary and if so use it  
 -   otherwise keep what you have

This works if we assume the sources were trying to do the right thing   
but their data is now out of date due to normal nomenclature drift.



In [119]:
path = "."
with open( path +'/previous_symbol_hgnc.yaml') as fh:
    string_symbol = yaml.load(fh)

In [132]:
s = 'BRCA1'
if s in string_symbol:
    print(string_symbol[s])
else:
    print(s)

s = 'FANCD'
if s in string_symbol:
    print(string_symbol[s])
else:
    print(s) 

BRCA1
FANCD2


In [ ]:
# make places to stash stuff 
genes = []
# tissues = []
# celltype = []
tissue_celltype = []
gene_tissue_curve = {}
oldsym_newsym = {}

# reduce ordered catagories to numbers 
levels = {
    'High' : 3,
    'Medium' : 2,
    'Low' : 1,
    'Not detected' : 0   
}

reliabilities = {
    'Approved' : 2,
    'Supported' : 1,
    'Uncertain' : 0
}

In [50]:
# AVOID rerunning this cell. it is __very__ expensive. 

file = 'normal_tissue.csv'
# "Gene|Gene name|Tissue|Cell type|Level|Reliability"

with open(file, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    csvreader.__next__()  # header
    for line in csvreader:
        # print(line)
        (gene, gene_name, tissue, cell_type, level, reliability) = line
        
        # bring gene symbols up to date
        #if gene_name not in oldsym_newsym:         
        #    symbol = symbol_check(gene_name)
        #    oldsym_newsym[gene_name] = symbol
        #else:
        #    symbol = oldsym_newsym[gene_name]
        
        if gene_name in string_symbol:
            symbol = string_symbol[gene_name]
        else:
            symbol = gene_name

        # keep the set of gene symbols
        if symbol not in genes:
            genes.append(symbol)
            if gene_name != symbol:
                print('NOTE: ' + symbol + ' is replacing ' + gene_name)
        # trim trailing ... study numbers?
        tissue = re.sub(r' [12]$', '', tissue)
        
        # keep the set of tissue+celltype keys
        tisscell = tissue + '|' + cell_type
        if tisscell not in tissue_celltype:
            tissue_celltype.append(tisscell)
            
        if symbol in gene_tissue_curve:
            gene_tissue_curve[symbol][tisscell] = levels[level]
        else:
            gene_tissue_curve[symbol] = {tisscell : levels[level]}

NOTE: MCUB is replacing CCDC109B
NOTE: ARL6IP6 is replacing AC004381.6
NOTE: TSPOAP1 is replacing BZRAP1
NOTE: REX1BD is replacing C19orf60
NOTE: ELOA is replacing TCEB3
NOTE: MRE11 is replacing MRE11A
NOTE: EIPR1-IT1 is replacing TSSC1
NOTE: RIPOR1 is replacing FAM65A
NOTE: RIPOR3 is replacing FAM65C
NOTE: HPF1 is replacing C4orf27
NOTE: BICRAL is replacing GLTSCR1
NOTE: INTS13 is replacing ASUN
NOTE: HDHD5-AS1 is replacing CECR5
NOTE: UFD1 is replacing UFD1L
NOTE: ELP1 is replacing IKBKAP
NOTE: BUD23 is replacing WBSCR22
NOTE: CARMIL1 is replacing LRRC16A
NOTE: FYB1 is replacing FYB
NOTE: C10orf10 is replacing RP5-1187M17.10
NOTE: ATP5A1P1 is replacing AC013461.1
NOTE: ADA2 is replacing CECR1
NOTE: WHRN is replacing DFNB31
NOTE: WASHC2A is replacing FAM21A
NOTE: ESS2 is replacing DGCR14
NOTE: GRK3 is replacing ADRBK2
NOTE: PAK5 is replacing PAK7
NOTE: RUBCNL is replacing KIAA0226L
NOTE: PYCR3 is replacing PYCRL
NOTE: CGB3 is replacing CGB
NOTE: DMAC2 is replacing ATP5SL
NOTE: NOP53 i

In [82]:
# save a copy , dont want to have to rerun that
with open('gene_tissue_curve.yaml', 'w') as fh:
    yaml.dump(gene_tissue_curve, fh)

In [51]:
len(gene_tissue_curve)

12935

In [52]:
# pairwise comparisons
len(gene_tissue_curve)*(len(gene_tissue_curve)-1)/2

83650645.0

In [53]:
len(gene_tissue_curve['FANCC'])

72

In [54]:
gene_tissue_curve['FANCC']

{'adrenal gland|glandular cells': 2,
 'appendix|glandular cells': 2,
 'bone marrow|hematopoietic cells': 2,
 'breast|adipocytes': 1,
 'breast|glandular cells': 2,
 'breast|myoepithelial cells': 2,
 'bronchus|respiratory epithelial cells': 2,
 'caudate|glial cells': 1,
 'caudate|neuronal cells': 1,
 'cerebellum|Purkinje cells': 2,
 'cerebellum|cells in granular layer': 2,
 'cerebellum|cells in molecular layer': 2,
 'cerebral cortex|endothelial cells': 0,
 'cerebral cortex|glial cells': 2,
 'cerebral cortex|neuronal cells': 1,
 'cerebral cortex|neuropil': 1,
 'cervix, uterine|glandular cells': 2,
 'cervix, uterine|squamous epithelial cells': 1,
 'colon|endothelial cells': 2,
 'colon|glandular cells': 2,
 'colon|peripheral nerve/ganglion': 1,
 'duodenum|glandular cells': 2,
 'endometrium|cells in endometrial stroma': 0,
 'endometrium|glandular cells': 2,
 'epididymis|glandular cells': 1,
 'esophagus|squamous epithelial cells': 2,
 'fallopian tube|glandular cells': 2,
 'gallbladder|glandul

removing the tissue population numbers "foo 1" & "foo 2" both become just "foo" means there can be > 1 xpat levels per gene-tissue

by default the last one wins, but that may not be the best stratagy  

TODO:  
revisit what to do with > 1 value per key
 - first
 - last
 - random
 - drop
 - average

In [56]:
len(tissue_celltype)

103

In [57]:
# want a more compact representation
gene_curves = {}
for gene in genes:
    gene_curves[gene] = array.array('B')
    for tissue in tissue_celltype:
        if tissue in gene_tissue_curve[gene]:
            gene_curves[gene].append(
                gene_tissue_curve[gene][tissue])
        else:
            gene_curves[gene].append(0)

In [58]:
str(gene_curves['FANCC'])

"array('B', [2, 2, 0, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 0, 2, 1, 1, 2, 1, 2, 2, 1, 2, 0, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 1, 0, 1, 0, 0, 0, 2, 1, 2, 2, 2, 1, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])"

Pairwise difference:  
 keep resulting vector?  
 or reduce to sum of difference? 

reducing to sum because  
a perfect match looks just like two not detecteds

since we know the total difference between 
not detected anywhere and high expression in every tissue. 
we can normalize the observed total difference. 

since we want to view less difference as better   
we can subtract the normalized difference from one  
so "bigger is gooder"(tm).


In [59]:
e2 = len(genes)-1
e1 = e2 - 1
crv_len = len(gene_curves[genes[0]])
result = []
associated = []
scale = 3 * crv_len
for g1 in range(e1):
    ac = gene_curves[genes[g1]]
    for g2 in range(g1+1, e2):
        bc = gene_curves[genes[g2]]
        r = 1 - (sum([abs(ac[j]-bc[j]) for j in range(crv_len)])) / scale
        result.append(r)
        if r == 1:
            associated.append([genes[g1], genes[g2]])        

In [60]:
len(result)

83637711

some summary statistics to help decide a cutoff

In [61]:
mu = statistics.mean(result)

mu = 0.6637721307871485

In [62]:
statistics.median(result)

0.7508090614886731

In [63]:
statistics.mode(result)

0.7928802588996764

if 0.79 is the most common value we may want to be north of that 

In [64]:
std = statistics.pstdev(result, mu)

In [65]:
cutoff = mu + std
cutoff

0.8484854252361468

At one standard divaitions above the mean,  
How many make the cutoff?

In [66]:
len([x for x in result if x > cutoff])

9446570

millions seems a bit too many to be useful to humans

In [67]:
len([x for x in result if x == 1])

27712

that is just about two associations per gene on averge  
which seems both reasonable and ultra conservative  

In [68]:
associated[0][0]

'CFH'

In [69]:
associated[0]

['CFH', 'KRIT1']

In [70]:
buffer = "graph {\n"
buffer += "\n".join(
    ['"' + a[0] + '" -- "' + a[1] + '" [weight="1"];' for a in associated]
) 
buffer += "\n}\n"

makeing pictures
```
    with open("hp_atlas_coxpat.gv",'w') as f:
        f.write(str(buffer))
``` 
should work in theroy but, is waaaaay too slow  

Dot must have an ```O(n^3)```  algo or worse  

In the past I have used CytoScape to viz hairballs
but it will have to wait to be installed on this machine.

Instead of trying to vizualize the clusters  
just (union) find them as connected components.  
NetworkX has that and more builtin.

In [71]:
# load the associated genes into an undirected graph
# I am not giving the edge weight here because it is a constant unit.
# if we back the cutoff down from perfects only  
# we will need to include the weight  
G = nx.Graph()
for e in associated:
    G.add_edge(e[0], e[1])

In [72]:
print(nx.number_connected_components(G))

218


 218 clusters is promising

In [78]:
print(str(len(genes)))
len(genes) / 218

12935


59.3348623853211

If uniform,   
means that on average a gene will be in clusters of about 60  
which if the co-expression method has any merit will display some cohesion.

Returning all the genes in a cluster is likely too many
but returning all that are one hop away seems reasonable.

In [74]:
# subgraphs = nx.connected_component_subgraphs(G)
# print(str(len(G.nodes())))
# print(G['SEPT14'])
G.neighbors('EGFL6')

['PLAC1',
 'FBN2',
 'CGB2',
 'PSG5',
 'CGB1',
 'GPR32',
 'PSG4',
 'PSG6',
 'VGLL3',
 'PSG7',
 'PSG9',
 'SIGLEC6',
 'HLA-G',
 'CGB8',
 'PSG8',
 'VGLL1',
 'CGB7',
 'PSG1',
 'PSG3',
 'HTRA4',
 'LVRN',
 'PSG11',
 'CGB5']

In [75]:
# what are all the clusters?
clusters = nx.connected_components(G)
# tl;dr the clustered list of genes looks very reasonable
# very frequently the majority of genes in a cluster 
# have the nomenclature of beloging to the same family
# 
# concerns: 
# may be a power distrabution 
# where there are a few clusters with many genes 
# and many clusters with few genes
# this could indicate a large cluster 
# is not particularly discriminating
# remember perfect matchs looks just like nothing detecteds
# maybe we can keep the genes total expression around to help ...

In [76]:
c = 0
for cluster in clusters:
    c+=1
    print("######################")
    for gene in cluster:
        print(str(c) +"\t" + gene)

######################
1	RGPD8
1	APOC4
1	RBM14-RBM4
1	APOH
1	NBPF6
1	COL8A2
1	EPB42
1	BDKRB1
1	ZNF583
1	CCM2L
1	PDE7B
1	DUXA
1	ANTXR1
1	GJD4
1	ODF3L2
1	HOXC13
1	USP17L8
1	BEST1
1	PCDH11Y
1	OR6T1
1	COL7A1
1	IL17C
1	N6AMT1
1	PWP2
1	TRMT10A
1	CNGB3
1	STAB2
1	MARCH3
1	MYOM3
1	APOA2
1	MED22
1	KAT14
1	SPEM2
1	AWAT1
1	FAM186B
1	C1S
1	PCSK4
1	FRMPD2
1	F5
1	RUNDC3B
1	WDR38
1	POTEH
1	SLC7A14
1	RADIL
1	CLEC4G
1	ZNF789
1	PPP1R27
1	EPHX3
1	MFAP2
1	MAPK8IP2
1	POTEJ
1	LRRC39
1	SLC17A2
1	MUC3A
1	CFH
1	HSFX1
1	SERPIND1
1	LRAT
1	C2
1	RNASE11
1	ZCWPW1
1	CLVS2
1	CER1
1	AFM
1	SIGLEC11
1	SYT16
1	CXorf66
1	BHLHE23
1	FN3K
1	KRIT1
1	ZFP64
1	SYNDIG1
1	RHAG
1	VENTX
1	YIPF7
1	METTL12
1	PGBD3
1	OR51J1
1	LHB
1	FGF7
1	FAM231D
1	OR1L3
1	PROK2
1	APOA1
1	KIAA0895
1	FAM47B
1	NODAL
1	CDC20B
1	A1BG
1	FOXD4
1	LILRB4
1	C5
1	KCTD19
1	AKAIN1
1	EPB41
1	ZSWIM5
1	NID1
1	EYS
1	SRRM3
1	FOXD4L4
1	DONSON
1	WDFY4
1	SMOC1
1	NANOG
1	CAPN7
1	RTL10
1	SLC44A5
1	C1orf68
1	FOXD4L5
1	COL3A1
1	FGG
1	ITIH2
1	HACD4
1	F12
1	CMTM1
1	FRG2B
1	ABCC9

get the FA genes   
adapt cells from CQ1.5

In [136]:
# because I put numbers in the filenames to order them
fa_sets={
    'core_complex' : {'num': '1'},
    'effector_proteins' :  {'num': '2'},
    'associated_proteins' : {'num': '3'}
}

# baseurl = 'https://raw.githubusercontent.com/NCATS-Tangerine/cq-notebooks/master/'
# Local copy
baseurl = '/home/tomc/Projects/OHSU/TransmedTranslator/cq-notebooks/'
# keep a handy list too
fagenes = []

for fa_set in fa_sets:
    path = baseurl + 'FA_gene_sets/FA_' + fa_sets[fa_set]['num'] + '_' + fa_set + '.txt'
    with open(path, 'r') as tabfile:
        filereader = csv.reader(tabfile, delimiter='\t')
        fa_sets[fa_set].pop('num') # was just there for file path
        for row in filereader:   
            (fa_curie, fa_symbol) = row
            if fa_symbol in string_symbol:
                fa_symbol = string_symbol[fa_symbol]
            fagenes.append(fa_curie)
            fa_sets[fa_set][fa_curie]= {
                    'symbol': fa_symbol, 
            }
    print('In the ' + fa_set + ' set we found  \t' + str(len(fa_sets[fa_set])) + ' genes')

In the effector_proteins set we found  	11 genes
In the core_complex set we found  	11 genes
In the associated_proteins set we found  	5 genes


In [140]:
'SAG' in G.nodes() 

True

In [137]:
for fa_set in fa_sets:
    for fagene in fa_sets[fa_set]:
        fa_sym = fa_sets[fa_set][fagene]['symbol']
        print(fa_sym)
        if fa_sym in G.nodes():
            for g in G.neighbors(fa_sym):
                print("\t", g)
   

RAD51
RFWD3
PALB2
SLX4
MAD2L2
ERCC4
BRCA2
BRIP1
BRCA1
RAD51C
XRCC2
FANCA
FANCL
UBE2T
FANCG
FANCF
FANCD2
FANCC
FANCM
FANCE
FANCI
FANCB
FAAP24
FAAP100
FAAP20
CENPX
CENPS


GAAAHHH!

13k genes with expression and not a single one from the FA sets.  
